# Priprema video zapisa za analizu 

## 1. Faza - podešavanja i odabir ulaznih video zapis te definiranje točaka rezanja

**Produkti faze:** `Ulaz_video`, `Rezovi_original.xlsx`  

Postupak formiranja datoteke `Rezovi_original.xlsx` :

U ovom koraku sam izabrao samo one video zapise, iz datoteka `Fine_aktivnosti` i `HE_aktivnosti`, koji
su sadržavali aktivnosti sastavljanja proizvoda tipa **T1**, **T2** i **T3**, ostali zapisi su zanemareni.

Definirao sam dokument `Rezovi_original.xlsx` u kojem su informacije o točkama rezanja pojedinih videa. Točke rezanja `Start` i `End` odabrane su tako da je u video zapisu koji rezultira rezanjem obuhvaćeno cijelo trajanje radnog ciklus operatera. Razlika između ove dvije točke zabilježena je u koloni `Duration`. Rezovi su ujedno i uzorci za daljnju obradu. 

Svakom od rezova (aka. uzoraka) pridodane su oznake:
* `Uzorak` - oznaka uzorka (#)
* `Day_ID` - dan prikupljanja videa (D#) 
* `Video_ID`- video zapis u danu (V#)
* `Subject` - operater koji izvodi aktivnosti (O#) 
* `Object_type` - tip proizvoda na kojem se radi aktivnost (T#)

Iz razloga što video kamera kojom su prikupljani uzorci podržava datoteke maksimalne veličine **3.5 GB**, a uzorci su prikupljani kontinuirano kroz cijelu radnu smjenu, trajanja 8 sati, moguće su situacije u kojima radni ciklus započinje u jednom videu, a završava u drugom (npr. V2 i V3). U ovom slučaju u tablici `Rezovi_original.xlsx` u koloni `Napomena` kod oba zapisa stavljena je oznaka **part**, a u koloni `Uzorak` prvi video ima isključivo numeričku oznaku, a drugi video istu numeričku oznaku ali i sufiks "A" (npr. 129 i 129A). Ovaj stil označavanja odabran jer jer će kasnije takvi video zapisi biti spojeni i činiti će jedan uzorak.

Finalno definirana je i kolona `Output_name`, koja sadrži konkatenaciju kolona sa oznakama (npr. **2_D1_V1_O1_T1**), ona će služiti za pridodavanje imena direktorijima koji će sadržavati izvučene slike iz video zapisa.

Postupak formiranja direktorija `Ulaz_video` :

`Ulaz video` sadržava u sebi tri datoteke `Fine`, `Fokus` i `HE`  u kojima se nalaze video zapisi snimljeni iz različitih kadrova.

* Kadar **Fine** snimljen je iz lijeve bočne pozicije operatera. Video zapisi smanjeni na rezoluciju "288x224", 30 fps.
* Kadar **HE** snimljen je iz ptičje perspektive te je na njemu vidljiva cijela radna površina operatere. Video zapisi smanjeni su na rezoluciju "320x192", 30 fps.
* Kadar **Fokus** snimljen je isto iz ptičje perspektive ali na njemu je fokus na proizvodu. Video zapisi smanjeni su na rezoluciju "320x160", 30 fps.




### Uvoz potrebnih biblioteka za sve faze

In [1]:
import os
import shutil
import pandas as pd
#samostalno kreirana biblioteka za rad sa video zapisima
from video_editing_utils import video_to_segment, concat_two_videos, video_to_image 
from tqdm import tqdm

## 2. Faza - izvlačenje rezanih segmenata iz video zapisa

**Produkti faze:** `Rezovi.xlsx`, `Video_rezovi`  

Ažurirane su kolone `Start`, `End` i `Duration` i napravljena je tablica `Rezovi.xlsx`.
Izrađena je skripta koja koriste podatke iz tablice `Rezovi.xlsx`, kako bi na odgovarajućim mjestima izvukla segmente video zapisa iz direktorija `Ulaz_video`. Rezovi su pohranjeni u direktoriju `Video_rezovi`. 

### Uvoz podataka o točkama rezanja

In [25]:
#Putanja do tablice "Rezovi"
cut_path = "C:/Users/Public/Rezovi.xlsx"

#Učitavanje podataka za kadrove HE i Fokus
rezovi_HE = pd.read_excel(cut_path)
#Učitavanje podataka za kadar Fine
rezovi_fine = pd.read_excel(cut_path, sheet_name=1)#to je na listu 2 (0-indeksiranje)

In [26]:
#Pregled uvezenih podataka - kadrovi HE i Fokus
rezovi_HE.head(8)

,Uzorak,Day_ID,Video_ID,Subject,Object_type,Output_name,Start,End,Duration,Napomena
0,1,D1,V1,O1,T1,1_D1_V1_O1_T1,09:18:00,10:12:00,00:54:00,NaN
1,2,D1,V1,O1,T1,2_D1_V1_O1_T1,10:12:00,10:57:00,00:45:00,NaN
2,3,D1,V1,O1,T1,3_D1_V1_O1_T1,10:57:00,11:36:00,00:39:00,NaN
3,4,D1,V1,O1,T1,4_D1_V1_O1_T1,11:36:00,12:28:00,00:52:00,NaN
4,5,D1,V1,O1,T1,5_D1_V1_O1_T1,12:28:00,13:18:00,00:50:00,NaN
5,6,D1,V1,O1,T1,6_D1_V1_O1_T1,13:18:00,14:09:00,00:51:00,NaN
6,7,D1,V1,O1,T1,7_D1_V1_O1_T1,14:09:00,15:01:00,00:52:00,NaN
7,8,D1,V1,O1,T1,8_D1_V1_O1_T1,15:01:00,15:47:00,00:46:00,NaN


In [4]:
#Tipovi podataka u kolonama
{kolona: type(rezovi_HE[kolona][0]) for kolona in rezovi_HE}

{'Uzorak': int,
 'Day_ID': str,
 'Video_ID': str,
 'Subject': str,
 'Object_type': str,
 'Output_name': str,
 'Start': datetime.time,
 'End': datetime.time,
 'Duration': datetime.time,
 'Napomena': float}

In [28]:
#Pretvorba kolona "Start", "End" i "Duration" u str, te u format <minuta:sekunda>
#ovo je potrebno jer funkcija video_to_segment kao ulaz prihvaća samo string, a ne datetime.time
#Ovdje koristim applymap, a ne apply jer radim transformaciju više kolona sa ne vektoriziranom funkcijom
rezovi_HE.loc[:, ["Start", "End", "Duration"]] = rezovi_HE.loc[:,["Start", "End", "Duration"]].astype("str").applymap(lambda x: x[:5])
rezovi_fine.loc[:, ["Start", "End", "Duration"]] = rezovi_fine.loc[:,["Start", "End", "Duration"]].astype("str").applymap(lambda x: x[:5])

In [6]:
rezovi_fine

,Uzorak,Day_ID,Video_ID,Subject,Object_type,Output_name,Start,End,Duration,Napomena
0,1,D1,V1,O1,T1,1_D1_V1_O1_T1,09:18,10:12,00:54,NaN
1,2,D1,V1,O1,T1,2_D1_V1_O1_T1,10:12,10:57,00:45,NaN
2,3,D1,V1,O1,T1,3_D1_V1_O1_T1,10:57,11:36,00:39,NaN
3,4,D1,V1,O1,T1,4_D1_V1_O1_T1,11:36,12:28,00:52,NaN
4,5,D1,V1,O1,T1,5_D1_V1_O1_T1,12:28,13:18,00:50,NaN
...,...,...,...,...,...,...,...,...,...,...
658,624,D7,V22,O3,T1,624_D7_V22_O3_T1,03:14,04:10,00:56,NaN
659,625,D7,V22,O3,T1,625_D7_V22_O3_T1,04:11,05:21,01:10,NaN
660,626,D7,V22,O3,T1,626_D7_V22_O3_T1,05:22,06:27,01:05,NaN
661,627,D7,V22,O3,T1,627_D7_V22_O3_T1,06:28,07:22,00:54,NaN


In [8]:
#Provjera da li je sve ok
sum(pd.isna(rezovi_fine.Duration)) == 0

True

### Definiranje putanja do datoteka sa ulaznim video zapisima za sve kadrove

In [29]:
#Putanja do glavnog direktorija
video_path = 'C:/Users/Public/Ulaz_video'

#Kadrovi kamere
kadrovi_path = os.listdir(video_path)

#Putanje do poddirektorija sa snimkama iz pojedinih kadrova
input_dir_fine = os.path.join(video_path, kadrovi_path[0])
input_dir_fokus = os.path.join(video_path, kadrovi_path[1])
input_dir_HE = os.path.join(video_path, kadrovi_path[2])

### Obrada HE kadra

In [30]:
#Pregled indeksa kolona - ovo je potrebno zbog poziva funkciji video_to_segment
for idx, rez in enumerate(rezovi_HE.columns):
    print(idx, "=>", rez)

0 => Uzorak
1 => Day_ID
2 => Video_ID
3 => Subject
4 => Object_type
5 => Output_name
6 => Start
7 => End
8 => Duration
9 => Napomena


In [11]:
#Definiranje funkcije za izvlačenje segmenata iz videa

def video_cutter(in_video_path, out_video_path, cut_data):
    '''Funkcija za izvlačenje segmenata iz svih videa jednog kadra za moj skup podataka.
    
    Da bi ova funkcija bila primjenjiva direktoriji i podatci o rezovima moraju biti
    organizirani kao što je opisano u 1. fazi
    
    in_video_path : apsolutna putanja do direktorija sa ulaznim video zapisima
    
    out_video_path : apsolutna putanja do direktorija za izvučene segmente
    
    cut_data : [array_like] tablica sa podacima o točkama rezanja
    
    Return : None
    '''
    #Petlja po danu zbog načina na koji je oblikovan input direktorij, tqdm je tu kao tajmer
    for day in tqdm(os.listdir(in_video_path)):
        #Petlja po videu u pojedinom danu
        for video in os.listdir(os.path.join(in_video_path, day)): 
            temp = video.split(".")[0] #izbaci ".mp4" zbog slijedećeg koraka
            #Informacija o rezovima za aktualni dan i video
            cuts = cut_data.loc[(cut_data["Day_ID"] == day) & (cut_data["Video_ID"] == temp)] 
            #Putanja do videa
            path_to_video = os.path.join(in_video_path, day, video) 
            #Petlja po izabranim rezovima
            for cut in cuts.values: 
                #Ovdje koristimo glavnu funkciju koja poziva ffmpeg za rezanje
                video_to_segment(path_to_video, out_video_path, output_file=f"{cut[5]}.mp4", start_point=cut[6], duration=cut[8])

In [12]:
#Definiranje izlazne datoteke za rezove snimaka iz kadra HE
output_dir_HE = "C:/Users/Public/Video_rezovi/HE"

#Izvlačenje segmenata za HE
video_cutter(input_dir_HE, output_dir_HE, rezovi_HE)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:04<00:00, 24.92s/it]


### Obrada Fokus kadra

In [13]:
#Definiranje izlazne datoteke za rezove snimaka iz kadra Fokus
output_dir_fokus = "C:/Users/Public/Video_rezovi/Fokus"

#Izvlačenje segmenata za Fokus, točke rezanja iste su kao za kadar HE
video_cutter(input_dir_fokus, output_dir_fokus, rezovi_HE)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:22<00:00, 28.49s/it]


### Obrada Fine kadra

In [14]:
#Definiranje izlazne datoteke za rezove snimaka iz kadra Fine
output_dir_fine = "C:/Users/Public/Video_rezovi/Fine"

#Izvlačenje segmenata za Fine, točke rezanja su "rezovi_fine"
video_cutter(input_dir_fine, output_dir_fine, rezovi_fine)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:28<00:00, 29.62s/it]


## 3. Faza - spajanje izvučenih segmenta koji čine isti ciklus

**Produkti faze:** Ažurirani `Video_rezovi`, `Rezovi_final.xlsx`

Nakon izvlačenja video segmenata iz dugih videa, postoji nekolicina videa koji čine jedan ciklus ali su podjeljeni u dva videa.
Ove video segmente moguće je prepoznati po oznaci **"part"** u koloni `Napomena`.
Slijedeći korak je pronaći listu tih videa i spojiti ih u jedan video i to zapisati u datoteku `Rezovi_final.xlsx`

### Video zapisi za kadrove Fokus i HE koje treba spojiti

In [15]:
#Odbaci NaN retke i odaberi za one retke koji sadrže u koloni Napomena => "part" oznaku
rezovi_HE_concat = rezovi_HE.dropna().loc[rezovi_HE.dropna()["Napomena"].str.contains("part", case=False)]
rezovi_HE_concat

,Uzorak,Day_ID,Video_ID,Subject,Object_type,Output_name,Start,End,Duration,Napomena
10,11,D1,V1,O1,T1,11_D1_V1_O1_T1,17:36,17:41,00:05,part1
11,11A,D1,V2,O1,T1,11A_D1_V2_O1_T1,00:00,00:49,00:49,part1
31,31,D1,V2,O1,T1,31_D1_V2_O1_T1,16:57,17:40,00:43,part2
32,31A,D1,V3,O1,T1,31A_D1_V3_O1_T1,00:00,00:01,00:01,part2
90,89,D1,V5,O1,T2,89_D1_V5_O1_T2,17:34,17:40,00:06,part3
...,...,...,...,...,...,...,...,...,...,...
619,589A,D7,V20,O3,T1,589A_D7_V20_O3_T1,00:00,00:21,00:21,part31
635,605,D7,V20,O3,T1,605_D7_V20_O3_T1,17:08,17:40,00:32,part32
636,605A,D7,V21,O3,T1,605A_D7_V21_O3_T1,00:00,00:34,00:34,part32
652,621,D7,V21,O3,T1,621_D7_V21_O3_T1,17:31,17:40,00:09,part33


### Video zapisi za kadar Fine koje treba spojiti

In [16]:
#Odbaci NaN retke i odaberi za one retke koji sadrže u koloni Napomena => "part" oznaku
rezovi_fine_concat = rezovi_fine.dropna().loc[rezovi_fine.dropna()["Napomena"].str.contains("part", case=False)]
rezovi_fine_concat

,Uzorak,Day_ID,Video_ID,Subject,Object_type,Output_name,Start,End,Duration,Napomena
20,21,D1,V2,O1,T1,21_D1_V2_O1_T1,13:09,13:16,00:07,part1
21,21A,D1,V3,O1,T1,21A_D1_V3_O1_T1,00:00,00:38,00:38,part1
37,37,D1,V3,O1,T1,37_D1_V3_O1_T1,12:44,13:16,00:32,part2
38,37A,D1,V4,O1,T1,37A_D1_V4_O1_T1,00:00,00:18,00:18,part2
67,66,D1,V5,O1,T2,66_D1_V5_O1_T2,12:37,13:16,00:39,part3
...,...,...,...,...,...,...,...,...,...,...
621,589A,D7,V20,O3,T1,589A_D7_V20_O3_T1,00:00,00:21,00:21,part33
637,605,D7,V20,O3,T1,605_D7_V20_O3_T1,17:08,17:40,00:32,part34
638,605A,D7,V21,O3,T1,605A_D7_V21_O3_T1,00:00,00:34,00:34,part34
654,621,D7,V21,O3,T1,621_D7_V21_O3_T1,17:31,17:40,00:09,part35


In [17]:
#Definiranje funkcije za spajanje videa
def video_concatenate(video_path, concat_data, keep_inputs=False, dest_inputs=None):
    '''Funkcija za spajanje svih parova video zapisa jednog kadra za moji skup podataka.
    
    Funkcija spojeni video stavlja na lokaciju gdje su bili pohranjeni ulazni video zapisi.
    Ulazni video zapisi se uklanjaju ili sele na novu lokaciju.
    
    Da bi ova funkcija bila primjenjiva direktoriji i podatci o rezovima moraju biti
    organizirani kao što je opisano u 1. fazi
    
    video_path : apsolutna putanja do direktorija sa ulaznim video zapisima, 
                 i lokacija na koju se pohranjuje spojeni video
    
    concat_data : [array_like] tablica sa podacima o tome koje video zapise spojiti
    
    keep_inputs : [bool] da li je potrebno zadržati kopiju ulaznih video zapisa, 
                 ako je "keep_inputs=True" potrebno je definirati lokaciju na koju će
                 ulazni video zapisi biti preseljeni u "dest_inputs"
    
    dest_inputs : apsolutna putanja na koju će ulazni video zapisi biti preseljeni
    
    Return : None
    '''
    #Iz razloga što uvijek 2 videa imaju istu oznaku dijela
    parts = concat_data["Napomena"].drop_duplicates().values 
    #Petlja po videima za spajanje
    for part in tqdm(parts):
        #Nazivi dva videa koji će biti spojeni - nalaze se u koloni "Output_name"
        zapisi_za_spajanje = concat_data.loc[concat_data["Napomena"] == part, "Output_name"].values 
        #Putanja do videa koji će biti spojeni - glavni direktoriji + naziv datoteke
        path_to_concats = [os.path.join(video_path, f"{zapis}.mp4") for zapis in zapisi_za_spajanje] 
        #Privremena lokacija spojenog videa - jer će imati isti naziv kao i original, pa ne mogu biti u istom direktoriju
        temp_loc_spoja = "C:/Users/Public/temp_" 
        #Spojeni video dobiva naziv originalnog videa
        ime_spoja = f"{zapisi_za_spajanje[0]}.mp4" 
        #Ovo je glavna funkcija koja koristi ffmpeg za spajanje 2 videa, ona i kreira output_dir ako ne postoji
        concat_two_videos(path_to_concats[0], path_to_concats[1], output_dir=temp_loc_spoja, output_file=ime_spoja)
        
        #Da li je potrebno zadržati ulazne video zapise
        if not keep_inputs:
            #Uklanjanje ulaznih videa koji su spajani
            [os.remove(path) for path in path_to_concats]  
        else:
            #Kreiranje lokacije na koju će ulazni video zapisi biti preseljeni, ako već ne postoji
            if not os.path.exists(dest_inputs):
                os.mkdir(dest_inputs)
            #Pomicanje ulaznih videa na novu lokaciju
            [shutil.move(path, dest_inputs) for path in path_to_concats]
        
        #Spojeni video selim u direktorij originala
        shutil.move(os.path.join(temp_loc_spoja, ime_spoja), video_path)
        #Ukloni privremenu datoteku
        shutil.rmtree(temp_loc_spoja)

### Obrada kadra HE

In [18]:
#Spajanje video zapisa i zadržavanje ulaznih video zapisa
dest_HE_inputs = "C:/Users/Public/HE_inputs"
video_concatenate(output_dir_HE, rezovi_HE_concat, keep_inputs=True, dest_inputs=dest_HE_inputs)

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [01:33<00:00,  2.84s/it]


### Obrada Fokus kadra

In [19]:
#Spajanje video zapisa i zadržavanje ulaznih video zapisa
dest_fokus_inputs = "C:/Users/Public/Fokus_inputs"
video_concatenate(output_dir_fokus, rezovi_HE_concat, keep_inputs=True, dest_inputs=dest_fokus_inputs)

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [01:58<00:00,  3.58s/it]


### Obrada Fine kadra

In [20]:
#Spajanje video zapisa i zadržavanje ulaznih video zapisa
dest_fine_inputs = "C:/Users/Public/Fine_inputs"
video_concatenate(output_dir_fine, rezovi_fine_concat, keep_inputs=True, dest_inputs=dest_fine_inputs)

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [01:50<00:00,  3.17s/it]


## 4. Faza  - izvlačenje sličica iz video segmenata

**Produkti faze:** `Uzorak_slike`

Napravljena je skripta koja koriste video zapise iz datoteke `Video_rezovi`, kako bi se provelo izvlačenje sličica iz video zapisa, uz korak uzorkovanja od **5 fps**, tj. u jednoj sekundi je uniformno uzorkovano 5 sličica, što znači da je preciznost kod označavanja aktivnosti ograničena na dvije desetinke sekunde (npr. 10.2, 11.4 itd.). 
Razlozi poduzorkovanja su praktične prirode - 1.kako bi model mogao biti naučen primjenom postojećih resursa, 2.postoji značajna redundantnost između susjednih sličica.

### Ulazni podatci 

In [35]:
#Putanje do direktorija sa video segmentima za obradu
#Ovo direktoriji izlaze prethodne faze, ovdje ih zovemo drugačije zbog jasnoće
HE_video_path = "C:/Users/Public/Video_rezovi/HE"
Fokus_video_path = "C:/Users/Public/Video_rezovi/Fokus"
Fine_video_path = "C:/Users/Public/Video_rezovi/Fine"

In [36]:
#Definiranje funkcije za izvlačenje sličica iz svih video zapisa jednog kadra
def image_extractor(in_video_path, out_image_path, image_res):
    '''Funkcija za izvlačenje sličica iz svih videa jednog kadra za moji skup podataka.
    
    Da bi ova funkcija bila primjenjiva direktoriji i podatci o rezovima moraju biti
    organizirani kao što je opisano u 1. fazi
    
    in_video_path : apsolutna putanja do direktorija sa ulaznim video zapisima
    
    out_image_path : apsolutna putanja do direktorija u kojem će biti poddirektoriji sličica pojedinog video zapisa
    
    image_res : <"širina:visina"> rezolucija slike
    
    Return : None
    '''
    #Petlja po video zapisima ulaznog direktorija
    for video in tqdm(os.listdir(in_video_path)):
        #Kreriranje putanje do ulaznog videa
        input_video = os.path.join(in_video_path, video)
        #Definiranje izlaznog poddirektorija - ovo je naziv direktorija sličica svakog pojedinog videa
        output_subfolder = video.split(".")[0] #izbaci ".mp4" dio
        #Glavna funkcija koja koristi ffmpeg za izvlačenje sličica iz video zapisa, uz uzorkovanje 5 sličica po sekundi
        video_to_image(input_video, os.path.join(out_image_path, output_subfolder), image_res=image_res, fps=5)

### Obrada kadra HE

In [23]:
#Putanja do izlaznog direktorija
HE_image_output = "C:/Users/Public/Uzorak_slika/HE"
HE_image_res = "320:192"
image_extractor(HE_video_path, HE_image_output, HE_image_res)

100%|████████████████████████████████████████████████████████████████████████████████| 628/628 [06:34<00:00,  1.59it/s]


### Obrada kadra Fokus

In [33]:
#Putanja do izlaznog direktorija
Fokus_image_output= "C:/Users/Public/Uzorak_slika/Fokus"
Fokus_image_res = "320:160"
image_extractor(Fokus_video_path, Fokus_image_output, Fokus_image_res)

100%|████████████████████████████████████████████████████████████████████████████████| 628/628 [05:46<00:00,  1.81it/s]


### Obrada kadra Fine

In [34]:
#Putanja do izlaznog direktorija
Fine_image_output = "C:/Users/Public/Uzorak_slika/Fine"
Fine_image_res = "288:224"
image_extractor(Fine_video_path, Fine_image_output, Fine_image_res)

100%|████████████████████████████████████████████████████████████████████████████████| 628/628 [06:41<00:00,  1.56it/s]


## 5. Faza  - preimenovanje poddirektorija u Uzorak_slika/Fine kako bi imale iste oznake kao i uzorci u Fokus i HE kadru

**Produkti faze:** Ažurirane oznake u  `Uzorak_slike/Fine`, ažurirani `Rezovi_final.xlsx`

Zbog načina na koji je sniman kadar Fine u **1. danu** dio oznaka uzoraka razlikuje se od onih koje imaju uzorci iz kadrova Fokus i HE, po pitanju Video_ID oznake (kadar fine ih ima više jer je bio sniman na 60fps-a). Ovo je problematično kod daljnje obrade po pitanju označavanja. Stoga će njihove oznake za 1. dan biti standardizirane, kako bi bile jednake onima kod ostala dva kadra.

In [22]:
#Putanje do Fine oznaka
fine_path = "C:/Users/Public/Uzorak_slika/Fine"

#Standardne oznake, kao u npr. HE kadru, za 1. dan
std_oznake = os.listdir("C:/Users/Public/Uzorak_slika/HE")
std_oznake_D1 = [oznaka for oznaka in std_oznake if oznaka.split("_")[1] == "D1"]

#Oznake kod Fine kadra, za 1. dan
fine_oznake = os.listdir("C:/Users/Public/Uzorak_slika/Fine")
fine_oznake_D1 = [oznaka for oznaka in fine_oznake if oznaka.split("_")[1] == "D1"]

#Sortiranje jednih i drugih oznaka prema oznaci uzorka
std_oznake_D1 = sorted(std_oznake_D1, key=lambda x: int(x.split("_")[0]))
fine_oznake_D1 = sorted(fine_oznake_D1, key=lambda x: int(x.split("_")[0]))

#Petlja po oznakama u kadru Fine i kadru HE
for std_lab, fine_lab in zip(std_oznake_D1, fine_oznake_D1):
    #Preimenovanje oznaka u Fine kadru sa imenima i kadra HE
    os.rename(src=os.path.join(fine_path, fine_lab), dst=os.path.join(fine_path, std_lab))


In [27]:
#Provjera da li su sada sve oznake iste
os.listdir("C:/Users/Public/Uzorak_slika/HE") == os.listdir("C:/Users/Public/Uzorak_slika/Fine")

True